In [32]:
from dotenv import load_dotenv
load_dotenv()


True

In [33]:
from langchain.chat_models import ChatOpenAI
import os


In [34]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


In [25]:
response = llm.predict("What is the capital of Andhra pradesh?")
print(response)


The capital of Andhra Pradesh is Amaravati.


In [51]:
import nest_asyncio
nest_asyncio.apply()


In [52]:
from playwright.async_api import async_playwright
import asyncio
import re
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage


In [53]:
def extract_site_from_instruction(instruction: str) -> str:
    known_sites = {
        "amazon": "https://www.amazon.in",
        "flipkart": "https://www.flipkart.com",
        "myntra": "https://www.myntra.com",
        "youtube": "https://www.youtube.com",
        "google": "https://www.google.com"
    }
    for keyword, url in known_sites.items():
        if keyword in instruction.lower():
            return url
    return "https://www.google.com"


In [54]:
async def extract_dom_elements(page, max_elements=25):
    elements = await page.query_selector_all("*")
    visible_elements = []

    for el in elements:
        try:
            tag = await el.get_property("tagName")
            tag = await tag.json_value()
            text = await el.inner_text()
            selector = await page.evaluate('(el) => el.outerHTML', el)
            if text.strip():
                visible_elements.append({
                    "tag": tag,
                    "text": text.strip(),
                    "html": selector[:120] + "..."  # Shortened
                })
            if len(visible_elements) >= max_elements:
                break
        except:
            continue

    return visible_elements


In [55]:
llm = ChatOpenAI(temperature=0, model="gpt-4o")  # Use gpt-3.5 if needed

async def run_web_task(task_instruction: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # 🌐 Step 1: Open site based on instruction
        site = extract_site_from_instruction(task_instruction)
        print(f"🌐 Navigating to: {site}")
        await page.goto(site)

        for step in range(3):  # Max 3 LLM-action loops
            print(f"\n🔁 Step {step + 1}")
            dom_snapshot = await extract_dom_elements(page)

            prompt = [
                SystemMessage(content="You are a web automation agent. Based on DOM and instruction, decide next action."),
                HumanMessage(content=f"Task: {task_instruction}\n\nDOM:\n{dom_snapshot}")
            ]

            action = llm.predict_messages(prompt).content
            print("🧠 LLM Action:", action)

            # Very basic parsing (to be improved)
            if "goto" in action:
                match = re.search(r"goto\(['\"](.+?)['\"]\)", action)
                if match:
                    await page.goto(match.group(1))
                    continue
            elif "click" in action and "text=" in action:
                match = re.search(r"text=['\"](.+?)['\"]", action)
                if match:
                    await page.click(f"text={match.group(1)}")
                    continue
            elif "type" in action:
                match = re.findall(r"type\(['\"](.+?)['\"], ['\"](.+?)['\"]\)", action)
                if match:
                    selector, value = match[0]
                    await page.fill(selector, value)
                    continue
            elif "return" in action and "href" in action:
                links = await page.query_selector_all("a")
                for link in links:
                    href = await link.get_attribute("href")
                    if href and "http" in href:
                        await browser.close()
                        return href
            
            await page.wait_for_timeout(2000)

        await browser.close()
        return "❌ Task limit exceeded or no valid action."


In [56]:
instruction = "Search for a brown Campus Sutra shirt on Myntra and return the product URL"
result = await run_web_task(instruction)
print("✅ Final Result:", result)


🌐 Navigating to: https://www.myntra.com


Error: Page.goto: net::ERR_HTTP2_PROTOCOL_ERROR at https://www.myntra.com/
Call log:
  - navigating to "https://www.myntra.com/", waiting until "load"
